In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic, Matern

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import r2_score

# user variables to tune
folds   = 10
scaler = MinMaxScaler(feature_range=(0, 1))

In [2]:
def predict(data,values,test,true,clf):
    model = clf
    train = model.fit(data,values)
    a = model.predict(test)
    df = pd.DataFrame(data=[a,true])
    
    return a,train,df

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

dataset = load_boston()
df = pd.DataFrame(dataset.data)
df.columns = dataset.feature_names
df["PRICE"] = dataset.target

X = df.drop("PRICE", axis=1)
Y = df["PRICE"]
print(X.shape)
print(Y.shape)

(506, 13)
(506,)


In [4]:
scaler.fit(X.values)
scaled_X = scaler.transform(X.values)

In [5]:
seed = 1000
test_size = 0.20

X_train, X_test, Y_train, Y_test = train_test_split(scaled_X, Y, test_size = test_size, random_state = seed)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [8]:
kernel = RationalQuadratic()

In [13]:
models = GaussianProcessRegressor(kernel)
# models = GaussianProcessRegressor()

In [15]:
pred,trained,df1 = predict(X_train, Y_train, X_test, Y_test, models)

In [17]:
display(df1)
print(models.score(scaled_X, Y))

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,21.33,16.17,16.12,38.43,23.81,33.91,12.83,35.68,19.37,16.55,...,25.06,11.71,25.25,22.01,20.18,25.92,20.45,20.64,20.84,20.61
1,21.80,10.20,15.20,43.50,50.00,37.20,7.00,42.30,16.00,19.60,...,24.80,12.30,24.60,24.30,19.00,22.00,20.00,19.90,19.10,21.20


0.956275858578696


In [18]:
# 10-fold cross validation for each model

k_fold  = KFold(n_splits=folds, random_state=seed)
results = cross_val_score(models, X_train, Y_train, cv=k_fold, scoring='r2')

print("r2:\t",results.mean(),"\nstd:\t",results.std())

r2:	 0.8130588660647001 
std:	 0.08591545149875958


In [14]:
print("initial params")
params = models.get_params()
print(params)

initial params
{'alpha': 1e-10, 'copy_X_train': True, 'kernel__length_scale': 1.0, 'kernel__alpha': 1.0, 'kernel__length_scale_bounds': (1e-05, 100000.0), 'kernel__alpha_bounds': (1e-05, 100000.0), 'kernel': RationalQuadratic(alpha=1, length_scale=1), 'n_restarts_optimizer': 0, 'normalize_y': False, 'optimizer': 'fmin_l_bfgs_b', 'random_state': None}


In [19]:
print("trained params")
params = trained.get_params()
print(params)

trained params
{'alpha': 1e-10, 'copy_X_train': True, 'kernel__length_scale': 1.0, 'kernel__alpha': 1.0, 'kernel__length_scale_bounds': (1e-05, 100000.0), 'kernel__alpha_bounds': (1e-05, 100000.0), 'kernel': RationalQuadratic(alpha=1, length_scale=1), 'n_restarts_optimizer': 0, 'normalize_y': False, 'optimizer': 'fmin_l_bfgs_b', 'random_state': None}
